In [1]:
import sys
sys.path.append('../../30_data_tools/')

In [2]:
import torch
from pathlib import Path
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from ultralytics import YOLO
from helper import load_dotenv
import pandas as pd
import shutil
from tqdm.auto import tqdm

In [3]:
dotenv = load_dotenv()

In [4]:
resnet_path = dotenv['MODEL_DIR'] / '24-02-24_01_resNet.pt'
yolo_path = dotenv['MODEL_DIR'] / 'yolov8_moires_24-02-22.pt'

In [5]:
resnet = torch.load( resnet_path )
yolo = YOLO(yolo_path)

In [6]:
resnet.eval()
transforms_data = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) # normalization
])

In [7]:
moire_paths = list(Path('./dataset/val/moire/').glob('./*.jpg'))
non_moire_paths = list(Path('./dataset/val/no_moire/').glob('./*.jpg'))

In [8]:
def predict_resnet( img_paths, model, transformation ):
    input_tensor = torch.stack([
        transforms_data(Image.open(img_path))
        for img_path in img_paths
    ])       
    output_tensor = model(input_tensor)

    out = output_tensor.tolist()
    for line in out:
        line.reverse()
        
    return out

In [9]:
def predict_yolo( img_paths, model ):
    images = [
        Image.open(img_path) for img_path
        in img_paths
    ]

    pred = model.predict(images, verbose=False)

    out = [p.probs.data.tolist() for p in pred]
    
    for line in out:
        line.reverse()

    return out

In [10]:
data = pd.DataFrame(
    [(t.name.replace(t.suffix,''),t,0) for t in non_moire_paths] + [(t.name.replace(t.suffix,''),t,1) for t in moire_paths],
    columns=['filename','filepath','target_label']
)

data.loc[
    :,
    ['resnet_no_moire','resnet_moire','yolo_no_moire','yolo_moire']
] = np.nan

In [11]:
with tqdm(total=data.shape[0]) as pbar:
    while data.loc[pd.isna(data.resnet_no_moire)].shape[0] > 0:
        remaining = data.loc[pd.isna(data.resnet_no_moire)].shape[0]
        n = 15 if remaining > 15 else remaining
        sample = data.loc[pd.isna(data.resnet_no_moire)].sample(n=n)
        yolo_result = predict_yolo( sample.filepath.tolist(), yolo )
        resnet_result = predict_resnet( sample.filepath.tolist(), resnet, transforms_data )
        
        data.loc[
            sample.index,
            ['resnet_no_moire','resnet_moire','yolo_no_moire','yolo_moire']
        ] = [
            (resnet_result[i][0],resnet_result[i][1],yolo_result[i][0],yolo_result[i][1])
            for i in range(len(resnet_result))
        ]

        remaining_after = data.loc[pd.isna(data.resnet_no_moire)].shape[0]
        pbar.update(remaining - remaining_after)

  0%|          | 0/574 [00:00<?, ?it/s]

In [12]:
data.loc[
    :,
    'resnet_label'
] = (data.resnet_moire > data.resnet_no_moire).astype('uint8')

data.loc[
    :,
    'yolo_label'
] = (data.yolo_moire > data.yolo_no_moire).astype('uint8')

In [13]:
data.shape

(574, 9)

In [14]:
sets_out = {
    'both_right' : data.loc[
        (data.target_label == data.resnet_label) &
        (data.target_label == data.yolo_label)
    ],
    'resnet_right' : data.loc[
        (data.target_label == data.resnet_label) &
        (data.target_label != data.yolo_label)
    ],
    'yolo_right' : data.loc[
        (data.target_label != data.resnet_label) &
        (data.target_label == data.yolo_label)
    ],
    'both_wrong' : data.loc[
        (data.target_label != data.resnet_label) &
        (data.target_label != data.yolo_label)
    ]
}

In [15]:
data.iloc[0]

filename           613256.tg46_132_133_JUL23_UPHK_WA_AS007_AT_DE_...
filepath           dataset/val/no_moire/613256.tg46_132_133_JUL23...
target_label                                                       0
resnet_no_moire                                             3.152367
resnet_moire                                               -3.155096
yolo_no_moire                                               0.999917
yolo_moire                                                  0.000083
resnet_label                                                       0
yolo_label                                                         0
Name: 0, dtype: object

In [16]:
(1093 + 43) / data.shape[0]

1.9790940766550522

In [17]:
for set_name in sets_out:
    print( set_name, sets_out[set_name].shape[0] )

both_right 467
resnet_right 35
yolo_right 26
both_wrong 46


In [18]:
for set_name in tqdm(sets_out):
    set_dir = Path(f'./temp/{ set_name }')

    if set_dir.exists() == False:
        set_dir.mkdir()

    for i in range(sets_out[set_name].shape[0]):
        row = sets_out[set_name].iloc[i]

        in_path = row.filepath
        type_name = 'moire' if row.target_label == 1 else 'no_moire'
        out_dir = set_dir / type_name

        if out_dir.exists() == False:
            out_dir.mkdir()
        
        out_path = out_dir / in_path.name

        shutil.copy(
            in_path,
            out_path
        )

  0%|          | 0/4 [00:00<?, ?it/s]